In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_auto_diffy")' SchwiftyNotebook_auto_diffy

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_auto_diffy")
		SchwiftyNotebook_auto_diffy
With SwiftPM flags: []
Working in: /tmp/tmp81k1ayqp/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information availa

In [2]:
//export
import TensorFlow
import SchwiftyNotebook_auto_diffy

In [3]:
import Python
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

public let plotter = Python.import("matplotlib.pyplot")

# Data Setup

In [6]:
let carDataYCSV = "/home/ubuntu/.machine-learning/data/car_stuff/craigslist_labels.csv"
let carDataXCSV = "/home/ubuntu/.machine-learning/data/car_stuff/craigslist_x_data.csv"

In [9]:
let craigslistVehiclesFull = Python.open(carDataXCSV)
print(Python.next(craigslistVehiclesFull))
print(Python.next(craigslistVehiclesFull))
print(Python.next(craigslistVehiclesFull))

,city,manufacturer,cylinders,fuel,paint_color,county_name,state_name,year,odometer,weather

0,0,0,0,0,0,0,0,2010.0,43600.0,38.0

1,0,1,1,0,1,1,0,1999.0,-1.0,38.0



In [11]:
let craigslistVehiclesLabel = Python.open(carDataYCSV)
print(Python.next(craigslistVehiclesLabel))
print(Python.next(craigslistVehiclesLabel))
print(Python.next(craigslistVehiclesLabel))

0,11900

1,1515

2,17550



In [10]:
public let numpy = Python.import("numpy")
public let pandas = Python.import("pandas")

In [13]:
Array(1...10)

▿ 10 elements
  - 0 : 1
  - 1 : 2
  - 2 : 3
  - 3 : 4
  - 4 : 5
  - 5 : 6
  - 6 : 7
  - 7 : 8
  - 8 : 9
  - 9 : 10


In [17]:
let carFeatures = numpy.loadtxt(carDataXCSV, 
                                delimiter: ",", 
                                skiprows: 1, 
                                usecols: Array(1...10), 
                                dtype: Float.numpyScalarTypes.first!)

In [20]:
let featuresTensor = Tensor<Float>(numpy: carFeatures) 

In [23]:
featuresTensor!.shape

▿ [1723065, 10]
  ▿ dimensions : 2 elements
    - 0 : 1723065
    - 1 : 10


In [24]:
let carPrices = numpy.loadtxt(carDataYCSV, 
                                delimiter: ",", 
                                skiprows: 0, 
                                usecols: [1], 
                                dtype: Float.numpyScalarTypes.first!)

In [25]:
let featuresTensor = Tensor<Float>(numpy: carPrices) 

In [26]:
featuresTensor?.shape

▿ Optional<TensorShape>
  ▿ some : [1723065]
    ▿ dimensions : 1 element
      - 0 : 1723065


In [36]:
let batchSize = 1028

struct CarBatch {
    let features: TensorFloat
    let labels: TensorFloat
}

extension CarBatch: TensorGroup {
    
    public static var _typeList: [TensorDataType] = [
        Float.tensorFlowDataType,
        Float.tensorFlowDataType
    ]
    public static var _unknownShapeList: [TensorShape?] = [nil, nil]
    public var _tensorHandles: [_AnyTensorHandle] {
        fatalError("unimplemented")
    }
    public func _unpackTensorHandles(into address: UnsafeMutablePointer<CTensorHandle>?) {
        address!.advanced(by: 0).initialize(to: features.handle._cTensorHandle)
        address!.advanced(by: 1).initialize(to: labels.handle._cTensorHandle)
    }
    public init(_owning tensorHandles: UnsafePointer<CTensorHandle>?) {
        features = Tensor(handle: TensorHandle(_owning: tensorHandles!.advanced(by: 0).pointee))
        labels = Tensor(handle: TensorHandle(_owning: tensorHandles!.advanced(by: 1).pointee))
    }
    public init<C: RandomAccessCollection>(_handles: C) where C.Element: _AnyTensorHandle {
        fatalError("unimplemented")
    }
}

In [37]:
extension Dataset where Element == CarBatch {
    init(featuresTensor: TensorFloat, labelsTensor: TensorFloat) {
        self.init(elements: CarBatch(features: featuresTensor, 
                                     labels: labelsTensor))
    }
}